# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [43]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import sqlite3

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to C:\Users\Captain
[nltk_data]     Nemo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Captain
[nltk_data]     Nemo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Captain Nemo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [44]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertDatabaseName', engine)
df.head(30)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2.0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7.0,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8.0,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9.0,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12.0,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14.0,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15.0,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16.0,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17.0,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18.0,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [45]:
# Assign message column as inputs and category columns as targets
X = df.message.values
y = df.iloc[:,4:].values

In [46]:
# checking unique values in y:
np.unique(y)

array([0, 1], dtype=int64)

### 2. Write a tokenization function to process your text data

In [47]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [36]:
# Split train and test datasets:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [7]:

# Training the pipeline:
pipeline.fit(X_train, y_train)
predicted = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Printing the classification report using scikit-learn method:
from sklearn.metrics import classification_report

# input datasets for classification report function should be dataframes:
category_names = df.iloc[:,4:].columns
y_test_df = pd.DataFrame(y_test, columns=category_names)
predicted_df = pd.DataFrame(predicted, columns = category_names)

print(classification_report(y_test_df, predicted_df, target_names= category_names))

                        precision    recall  f1-score   support

               related       0.66      0.87      0.75      1688
               request       0.28      0.02      0.03       908
                 offer       0.00      0.00      0.00         4
           aid_related       0.23      0.01      0.03       972
          medical_help       0.00      0.00      0.00       137
      medical_products       0.00      0.00      0.00        77
     search_and_rescue       0.00      0.00      0.00        44
              security       0.00      0.00      0.00        35
              military       0.00      0.00      0.00         7
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       183
                  food       0.00      0.00      0.00       384
               shelter       0.00      0.00      0.00       275
              clothing       0.00      0.00      0.00        21
                 money       0.00      

E:\Software\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Software\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Software\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Software\anaconda3\lib\site-packages\sklearn\metrics\_classification.

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {
    'vect__ngram_range': ((1,1),(1,2)),
    'clf__estimator__leaf_size': [30,40,50],
    'clf__estimator__n_neighbors':[5,7,9]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
predicted = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'clf__estimator__leaf_size': 30, 'clf__estimator__n_neighbors': 5, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
predicted_df = pd.DataFrame(predicted, columns = category_names)
print(classification_report(y_test_df, predicted_df, target_names= category_names))

                        precision    recall  f1-score   support

               related       0.67      0.89      0.76      1688
               request       0.25      0.00      0.01       908
                 offer       0.00      0.00      0.00         4
           aid_related       0.12      0.00      0.00       972
          medical_help       0.00      0.00      0.00       137
      medical_products       0.00      0.00      0.00        77
     search_and_rescue       0.00      0.00      0.00        44
              security       0.00      0.00      0.00        35
              military       0.00      0.00      0.00         7
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       183
                  food       0.00      0.00      0.00       384
               shelter       0.00      0.00      0.00       275
              clothing       0.00      0.00      0.00        21
                 money       0.00      

In [12]:
accuracy = 100 * (y_test==predicted).mean()
print("Accuracy: ", accuracy)

Accuracy:  92.87949921752738


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [48]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        pos_tags = []
        for sentence in sentence_list:
            pos_tags.append(nltk.pos_tag(tokenize(sentence)))
            first_word, first_tag = pos_tags[0][0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [41]:
X_train[50]

'i agree with everything the ministry of health says. but i have problmes finding food and water that is more urgent. its there that we need urgent help. muller avenue carrefour'

In [49]:
from sklearn.tree import DecisionTreeClassifier
pipeline = Pipeline([
    ('Features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor())
    ])),
    
    ('clf', DecisionTreeClassifier())
])
pipeline.fit(X_train, y_train)
predicted = pipeline.predict(X_test)
predicted_df = pd.DataFrame(predicted, columns = category_names)
print(classification_report(y_test_df, predicted_df, target_names= category_names))

IndexError: list index out of range

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.